In [1]:
import pandas as pd
data=pd.read_csv('output_data.csv')


In [2]:
d2 = data.dropna()

In [27]:
bin(d2.iloc[1]['TLS option 3'])[2:].zfill(32)


'00000000000000001100000000100111'

In [31]:
import pandas as pd
import ray
ray.shutdown()
ray.init()

@ray.remote
def update_df(dframe1):
    dframe2 = pd.DataFrame(columns=[f'tls option 1_{i}' for i in range(32)] + [f'tls option 2_{i}' for i in range(32)] + [f'dns option 1_{i}' for i in range(32)] + [f'dns option 2_{i}' for i in range(32)] + [f'http option 1_{i}' for i in range(32)] + [f'http option 2_{i}' for i in range(32)] + ['label'])
    
    for idx in range(len(dframe1)):
        da = dframe1.iloc[idx]
        packet_type = da['Packet Type']
        row = {}
        
        if packet_type == "TLS":
            if int(da['TLS option 2']) != -1:
                val = bin(int(da['TLS option 2']))
                bin_str = val[2:].zfill(32)
                row.update({f'tls option 1_{i}': int(bin_str[i]) for i in range(32)})
                row.update({f'tls option 2_{i}': -1 for i in range(32)})
            else:
                val = bin(int(da['TLS option 3']))
                print(val)
                bin_str = val[2:].zfill(32)
                print(bin_str)
                row.update({f'tls option 2_{j}': int(bin_str[j]) for j in range(32)})
                row.update({f'tls option 1_{i}': -1 for i in range(32)})
            row.update({f'dns option 1_{i}': -1 for i in range(32)})
            row.update({f'dns option 2_{i}': -1 for i in range(32)})
            row.update({f'http option 1_{i}': -1 for i in range(32)})
            row.update({f'http option 2_{i}': -1 for i in range(32)})
            
        elif packet_type == "DNS":
            if int(da['DNS TTL']) != -1:
                val = bin(int(da['DNS TTL']))
                bin_str = val[2:].zfill(32)
                row.update({f'dns option 1_{i}': int(bin_str[i]) for i in range(32)})
                row.update({f'dns option 2_{i}': -1 for i in range(32)})
            else:
                val = bin(int(da['DNS Donamin Length']))
                bin_str = val[2:].zfill(32)
                row.update({f'dns option 2_{i}': int(bin_str[i]) for i in range(32)})
                row.update({f'dns option 1_{i}': -1 for i in range(32)})
            row.update({f'tls option 1_{i}': -1 for i in range(32)})
            row.update({f'tls option 2_{i}': -1 for i in range(32)})
            row.update({f'http option 1_{i}': -1 for i in range(32)})
            row.update({f'http option 2_{i}': -1 for i in range(32)})
            
        elif packet_type == "HTTP":
            if da['HTTP Return Code'] == -1:
                val = bin(len(da['HTTP Content Type']))
                bin_str = val[2:].zfill(32)
                row.update({f'http option 1_{i}': int(bin_str[i]) for i in range(32)})
                row.update({f'http option 2_{i}': -1 for i in range(32)})
            else:
                val = bin(int(da['HTTP Return Code']))
                bin_str = val[2:].zfill(32)
                row.update({f'http option 2_{i}': int(bin_str[i]) for i in range(32)})
                row.update({f'http option 1_{i}': -1 for i in range(32)})
            row.update({f'tls option 1_{i}': -1 for i in range(32)})
            row.update({f'tls option 2_{i}': -1 for i in range(32)})
            row.update({f'dns option 1_{i}': -1 for i in range(32)})
            row.update({f'dns option 2_{i}': -1 for i in range(32)})
            
        else:
            continue
        
        row.update({'label':da['label']})
        
        dframe2 = dframe2.append(row, ignore_index=True)
        
        if idx+1 == 1000:
            return dframe2
    return dframe2

r = update_df.remote(d2)

df_1 = ray.get(r)
ray.shutdown()


2023-04-25 19:54:17,569	INFO worker.py:1553 -- Started a local Ray instance.


RayTaskError(ValueError): [36mray::update_df()[39m (pid=74373, ip=192.168.245.105)
  File "/tmp/ipykernel_11338/2874410448.py", line 26, in update_df
  File "/tmp/ipykernel_11338/2874410448.py", line 26, in <dictcomp>
ValueError: invalid literal for int() with base 10: 'b'

(update_df pid=74373) 0b1100000000100111
(update_df pid=74373) 00000000000000001100000000100111
(update_df pid=74373) -0b1
(update_df pid=74373) 000000000000000000000000000000b1


In [7]:
import pandas as pd
import ray
#ray.shutdown()
ray.init()

@ray.remote
def update_df(dframe1):
    dframe2 = pd.DataFrame(columns=[f'packet length {i}' for i in range(32)])
    for idx in range(911737):
        da = dframe1.iloc[idx]
        length = da['TLS option 1']
        len_bin = bin(length)
        bin_string = len_bin[2:].zfill(32)
        row = {f'packet length {i}': int(bin_string[i]) for i in range(32)}
        #row['Label'] = da['Label']
        dframe2 = dframe2.append(row, ignore_index=True)
        if (idx+1) % 10000 == 0:
            return dframe2
    return dframe2

r = update_df.remote(d2)

df_1 = ray.get(r)
ray.shutdown()


2023-04-25 19:13:43,478	INFO worker.py:1553 -- Started a local Ray instance.


In [8]:
df_1

,packet length 0,packet length 1,packet length 2,packet length 3,packet length 4,packet length 5,packet length 6,packet length 7,packet length 8,packet length 9,...,packet length 22,packet length 23,packet length 24,packet length 25,packet length 26,packet length 27,packet length 28,packet length 29,packet length 30,packet length 31
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,1,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,1,1,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,1
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
